In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

# get tfidf

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/IDClassifier/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(1575, 2)


,label,split_text
0,0,啊
1,1,你 有病 吧 我 跟 你 说 过 很 多次 了 我 不是...
2,1,不是 什么 事
3,0,啊 对
4,0,对
5,0,对
6,1,我 不是 啊
7,0,噢
8,0,啊
9,0,啊


In [3]:
# phrase_vectorizer1 = TfidfVectorizer(ngram_range=(1,3),
#                                     strip_accents='unicode', 
#                                     max_features=100000, 
#                                     analyzer='word',
#                                     sublinear_tf=True,
#                                     token_pattern=r'\w{1,}')
# data.split_text.iloc[:2]
# dd = ['I like apple do you like it', 'I do not like banana']
# phrase_vectorizer1.fit(dd)

# print('transform phrase')
# phrase = phrase_vectorizer1.transform(dd)


In [4]:
data.split_text.iloc[1]

'你   有病   吧   我   跟 你   说 过   很 多次   了   我   不是   他'

In [5]:
# phrase_vectorizer1.vocabulary_

In [6]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<1575x3941 sparse matrix of type '<class 'numpy.float64'>'
	with 15419 stored elements in Compressed Sparse Row format>

# SVC

In [7]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.969523809524


In [8]:
print(clf.predict(phrase))

[0 1 1 ..., 1 2 2]


# logistic

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.930158730159


In [11]:
print(log_r.predict(phrase))

[0 1 1 ..., 1 0 2]


In [12]:
# l_svc.predict(phrase)-log_r.predict(phrase)

# lightGBM

In [13]:
import lightgbm as lgb

In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [15]:
onelabels

array([0, 1, 1, ..., 1, 2, 2])

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [17]:
lables

array([[1, 1, 1]])

In [18]:
data.label.values

array([0, 1, 1, ..., 1, 2, 2])

In [19]:
# lables.todense()

In [20]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.16381
[10]	valid_0's multi_error: 0.144127
[15]	valid_0's multi_error: 0.12381
[20]	valid_0's multi_error: 0.111746
[25]	valid_0's multi_error: 0.0977778
[30]	valid_0's multi_error: 0.0895238
[35]	valid_0's multi_error: 0.0888889
[40]	valid_0's multi_error: 0.0844444
[45]	valid_0's multi_error: 0.0825397
[50]	valid_0's multi_error: 0.0806349
[55]	valid_0's multi_error: 0.0774603
[60]	valid_0's multi_error: 0.0768254
[65]	valid_0's multi_error: 0.0761905
[70]	valid_0's multi_error: 0.0749206
[75]	valid_0's multi_error: 0.0749206
[80]	valid_0's multi_error: 0.0736508
[85]	valid_0's multi_error: 0.072381
[90]	valid_0's multi_error: 0.072381
[95]	valid_0's multi_error: 0.072381
[100]	valid_0's multi_error: 0.071746
[105]	valid_0's multi_error: 0.0692063
[110]	valid_0's multi_error: 0.0692063
[115]	valid_0's multi_error: 0.0692063
[120]	valid_0's multi_error: 0.0692063
[125]	valid_0's multi_error: 0.

In [21]:
print(lgbm_model.predict(phrase))

[[  9.90568070e-01   4.83210937e-03   9.34213248e-03]
 [  5.35080526e-04   9.98512677e-01   2.50031682e-03]
 [  2.19988812e-02   9.84001859e-01   9.91147251e-03]
 ..., 
 [  6.63365535e-03   9.91822966e-01   3.98196403e-03]
 [  6.47736122e-01   1.14001591e-01   1.80868007e-01]
 [  2.40402187e-01   5.75542830e-03   7.78350920e-01]]


In [22]:
# lgbm_model.predict(phrase)

In [23]:
phrase

<1575x3941 sparse matrix of type '<class 'numpy.float64'>'
	with 15419 stored elements in Compressed Sparse Row format>

# Save

In [24]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/IDClassifier/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/IDClassifier/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/IDClassifier/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/IDClassifier/Lgbm.pickle", "wb"))

In [25]:
# pickle.load(open("../../savedModel/IDClassifier/Lgbm.pickle", 'rb'))

# test

In [27]:
test = phrase_vectorizer.transform(['是 呀'])
# test = phrase_vectorizer.transform(['我 在 洗澡'])


In [28]:
test

<1x3941 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [29]:
clf.predict_proba(test) # linear svc

array([[ 0.90866398,  0.03529406,  0.05604196]])

In [30]:
log_r.predict_proba(test) # logistic

array([[ 0.74566077,  0.14996941,  0.10436982]])

In [31]:
lgbm_model.predict(test) # light gbm

array([[ 0.97235174,  0.0247113 ,  0.01844675]])

# labeling

In [32]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 0
prob= 0.972351739991


# Save the whole thing

In [86]:
# import jieba
# import numpy as np

# class IDClassifier:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred<threshold):
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [87]:
idc.classify('我在洗澡')

(2, 0.95201366423625655)

In [88]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
from IDClassifier_py import IDClassifier
idc = IDClassifier(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')

pickle.dump(idc, open("../../savedModel/IDClassifier/IDClassifier.pickle", "wb"))

In [90]:
## when apply, need to import sys path of the .py file, then load pkl file 